# Notebook to explore the naming of the class names for FoodVision

* **Goal:** Get the class names for FoodVision and see which ones can be broken down into something more specific.


In [1]:
# Append the upper level directory to sys
import sys
sys.path.append("..")

import pandas as pd
import numpy as np

from pathlib import Path

import torch

from utils.misc import sort_dict_by_values

from zero_shot_classification import open_clip_compute_image_features_of_list, open_clip_compute_text_features, open_clip_zero_shot_classification


/home/daniel/.local/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/daniel/.local/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowIlEET_S2_S2_b
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# Get original labels to label new data from
# Get config
from configs.default_config import config

args = config

# Connect to GCP
from utils.gcp_utils import set_gcp_credentials, test_gcp_connection
set_gcp_credentials(path_to_key="../utils/google-storage-key.json")
test_gcp_connection()

import wandb

# Initialize a new run
from utils.wandb_utils import wandb_load_artifact, wandb_download_and_load_labels

notes = f"autolabel new images"

run = wandb.init(project=args.wandb_project, 
                 job_type=args.wandb_job_type,
                 tags=['auto_label_new_images'],
                 notes=notes)

annotations, class_names, class_dict, reverse_class_dict, labels_path = wandb_download_and_load_labels(wandb_run=run,
wandb_labels_artifact_name=args.wandb_labels_artifact)

len(annotations.label.unique())

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


[INFO] GCP credentials set!
[INFO] GCP connection successful! Access to GCP for saving/loading data and models available.


wandb: Currently logged in as: mrdbourke. Use `wandb login --relogin` to force relogin


[INFO] Labels directory: ./artifacts/food_vision_labels:v28
[INFO] Labels path: artifacts/food_vision_labels:v28/annotations.csv
[INFO] Working with: 209 classes


209

## Import VegFru class names

Source: https://docs.google.com/spreadsheets/d/1AbDDAuntzaI140wf5nAAgTbXe8thKJLr5E2ZbTlurbg/edit#gid=0

In [4]:
vegfru = pd.read_csv("nutrify-vegfru-dataset-class-crossover.csv")
print(f"Num rows: {len(vegfru)}")
vegfru.head()

Num rows: 293


,class_name,in_nutrify,nutrify_class_name,category,notes,Link to vegfru dataset -- https://www.kaggle.com/datasets/zhaoyj688/vegfru
0,almond,1.0,almonds,fruit,NaN,NaN
1,apple,1.0,apple_red,fruit,NaN,NaN
2,apricot,1.0,apricot,fruit,NaN,NaN
3,asparagus,1.0,asparagus,vegetable,NaN,NaN
4,avocado,1.0,avocado,fruit,NaN,NaN


In [5]:
# Drop rows if the nutrify_class_name is nan
vegfru = vegfru.dropna(subset=['nutrify_class_name'])
print(f"Num rows: {len(vegfru)}")

Num rows: 97


In [7]:
# Get the unique class names
vegfru_nutrify_unique_class_names = vegfru.nutrify_class_name.unique()

# Compare the unique class names to the class names in the annotations
# nutrify_class_names_not_in_vegfru = [class_name for class_name in annotations.label.unique() if class_name not in vegfru_nutrify_unique_class_names]
# nutrify_class_names_in_vegfru = [class_name for class_name in annotations.label.unique() if class_name in vegfru_nutrify_unique_class_names]
vegfru_class_names_not_in_nutrify = sorted([class_name for class_name in vegfru_nutrify_unique_class_names if class_name not in class_names])
vegfru_class_names_not_in_nutrify

['basil',
 'cabbage_red',
 'cherry_tomato',
 'chives',
 'cos_lettuce',
 'edamame',
 'endive',
 'hazelnuts',
 'honeydew_melon',
 'longan',
 'macadamia',
 'mint',
 'mushroom_oyster',
 'nectarine',
 'okra',
 'pear_nashi',
 'pecans',
 'persimmon',
 'pistachio',
 'radish_red',
 'radish_white',
 'swiss_chard',
 'walnuts',
 'watercress']